In [16]:
from opensearchpy import OpenSearch

index_name = "fed-court-chunks"


# Connect to your cluster
opensearch_client = OpenSearch(
    hosts=[{"host": "opensearch-dev", "port": 9200}],  # adapt to your setup
    http_compress=True
)

import pandas as pd
df = pd.read_parquet('../data/bger-2024-3-text.parquet')


In [17]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

test_cases = [
  {
    "title": "Litige transfrontalier en matière de protection des données",
    "content": "Une entreprise suisse basée à Zurich a transféré des données personnelles de ses clients vers un prestataire basé en Floride. Un client suisse s'est plaint que ses données aient été utilisées à des fins de marketing sans consentement explicite. L’affaire soulève la question de la conformité au droit suisse sur la protection des données et au principe du transfert international."
  },
  {
    "title": "Discrimination dans un marché réglementé",
    "content": "Un ingénieur software demandeur d'emploi de nationalité allemande affirme que sa candidature a été rejetée par une entreprise semi-publique suisse en raison de sa nationalité. Il invoque une discrimination contraire à l’accord sur la libre circulation des personnes et aux dispositions suisses sur l’égalité de traitement."
  },
  {
    "title": "Droit des brevets et concurrence déloyale",
    "content": "Une PME suisse accuse une entreprise concurrente d’avoir copié un procédé breveté dans le domaine de la chimie verte, utilisé dans toute la Suisse. Elle saisit le Tribunal fédéral des brevets pour violation de brevet, demandant l’interdiction de vente et des dommages-intérêts."
  },
  {
    "title": "Responsabilité de l’État en matière de cybersécurité",
    "content": "Un citoyen suisse voit ses données fiscales compromises après une cyberattaque sur un portail cantonal de déclaration en ligne. Il intente une action contre la Confédération, affirmant une responsabilité subsidiaire pour défaut de surveillance ou standards insuffisants de cybersécurité imposés aux cantons."
  },
  {
    "title": "Recours contre une décision de l’OFSP",
    "content": "Un thérapeute en médecine complémentaire attaque une décision de l’Office fédéral de la santé publique (OFSP) lui refusant l'autorisation de facturer via l’assurance de base (LAMal). Il estime que la décision viole son droit de pratiquer et demande au Tribunal administratif fédéral d’annuler la décision."
  },
  {
    "title": "Délit d’initié et marchés financiers",
    "content": "Un cadre supérieur d’une banque zurichoise est poursuivi pour avoir vendu des actions la veille d’une annonce de fusion confidentielle. La FINMA a transmis le dossier au Ministère public de la Confédération pour soupçon de délit d’initié, en application de la Loi sur les bourses (LBVM)."
  },
  {
    "title": "Asile et détention administrative",
    "content": "Un ressortissant érythréen, débouté de sa demande d’asile, fait recours contre sa mise en détention en vue du renvoi, décidée par le SEM. Il soutient que son renvoi est impossible et que sa détention viole l’article 5 CEDH et la jurisprudence fédérale sur la proportionnalité."
  },
  {
    "title": "Affaire douanière et taxation à l'importation",
    "content": "Une société suisse importe des pièces électroniques de Chine via un entrepôt sous douane à Bâle. L’Administration fédérale des douanes lui inflige une amende pour fausse déclaration de valeur. L’entreprise fait recours en affirmant avoir suivi les instructions d’un transitaire autorisé."
  },
  {
    "title": "Liberté religieuse et service civil",
    "content": "Un objecteur de conscience témoigne que le refus de son admission au service civil, malgré ses convictions religieuses pacifistes, viole sa liberté de conscience garantie par la Constitution fédérale. L’affaire est portée devant le Tribunal fédéral."
  },
  {
    "title": "Litige en matière d’accès à l’information",
    "content": "Un journaliste demande à l’OFEV l’accès à un rapport interne sur des risques environnementaux liés à une entreprise de recyclage. Le département refuse en invoquant des raisons de confidentialité industrielle. Le journaliste saisit le Préposé fédéral à la transparence puis fait recours devant le Tribunal administratif fédéral."
  }
]


case_number = 1
query_embedding = model.encode([test_cases[case_number]["content"]])[0]  # shape: (384,)

In [20]:
top_matches = 10
response = opensearch_client.search(
    index=index_name,
    body={
        "size": top_matches,
        "query": {
            "knn": {
                "embedding": {
                    "vector": query_embedding.tolist(),
                    "k": top_matches
                }
            }
        }
    },
    timeout=30
)
hit_ids = set()
for hit in sorted(response["hits"]["hits"], key=lambda x: x["_source"]["doc_id"], reverse=True):
    hit_ids.add(hit["_source"]["doc_id"])
    print(hit["_source"]["doc_id"], hit["_source"]["text"], hit["_score"])


8C_931/2009 2 OAFam, ou, à tout le moins, qu'elle remplit toutes les conditions pour être reconnue dans ce canton. 6.8 Il s'ensuit que le recours doit être admis en tant qu'il se fonde sur l'art. 49 Cst., ce qui rend superflu l'examen du second grief soulevé par la recourante et tiré de la garantie de la liberté économique. 7. La recourante, qui obtient gain de cause, a droit à des dépens, à la charge du canton du Valais (art. 68 al. 1 LTF). Conformément à l'art. 66 al. 4 LTF, il n'est pas perçu de frais judiciaires à la charge du canton. Par ces motifs, le Tribunal fédéral prononce: 1. Le recours est admis et le jugement attaqué est annulé. X.________ SA est autorisée à changer de caisse d'allocations familiales pour tous ses salariés exerçant une activité en Valais dès le 1er janvier 2009. 2. Il n'est pas perçu de frais judiciaires. 3. Une indemnité de 2'800 fr. à payer à la recourante à titre de dépens est mise à la charge du canton du Valais. 4. Le présent arrêt est communiqué aux 

In [30]:
relevant_docs = [
    {"doc": doc, "ref": doc_ref}
    for doc, doc_ref in zip(
        df.loc[df["docref"].isin(hit_ids), "text"],
        df.loc[df["docref"].isin(hit_ids), "docref"]
    )
]
relevant_docs

[{'doc': "Bundesgericht Tribunal fédéral Tribunale federale Tribunal federal {T 0/2} 8C_931/2009 Arrêt du 7 mai 2010 Ire Cour de droit social Composition MM. et Mmes les Juges Ursprung, Président, Leuzinger, Frésard, Niquille et Maillard. Greffière: Mme Hofer. Participants à la procédure X.________ SA, représentée par Me Denis Weber, avocat, recourante, contre Service cantonal des allocations familiales, avenue Pratifori 22, 1950 Sion, intimé. Objet Allocation familiale, recours contre le jugement du Tribunal cantonal valaisan, Cour des assurances sociales, du 28 septembre 2009. Faits: A. La société X.________ SA est affiliée depuis 1958 à la Caisse AVS de la Fédération patronale Vaudoise, à Y.________. Pour le régime des allocations familiales en faveur de ses salariés occupés en Valais, X.________ SA était affiliée depuis le 1er janvier 1997 à la Caisse interprofessionnelle valaisanne d'allocations familiales (ci-après: CIVAF). Par écriture du 26 juin 2008, elle a annoncé à la CIVAF 

In [ ]:
import openai
import os

api_key = os.environ["OPENAI_API_KEY"]

oai_client = openai.OpenAI(api_key=api_key)
# Set the OpenAI API key
question = f"Fournis un avis juridique en te fondant sur les documents joints concernant le cas suivant : {test_cases[case_number]['content']}"

# Combine the documents into a single prompt
context = "\n\n".join([f"Document {doc['ref']}:\n{doc['doc']}" for doc in relevant_docs])

response = oai_client.chat.completions.create(
    model="gpt-4.1-mini",
    messages=[
        {"role": "system", "content": "You are a legal assistant who answers based only on provided documents."},
        {"role": "user", "content": f"{context}\n\nQuestion: {question}"}
    ],
    temperature=0
)

print(response.choices[0].message.content)
